## 层和块

*块*(block)可以描述单个层、由多个层组成的组件或整个模型本身。使用块进行抽象的一个好处是可以将一些块组合成更大的组件，这一过程通常是递归的。   
从编程的角度来看，块由*类*(class)表示。它的任何子类都必须定义为一个将其输入转换成输出的前向传播函数，并且必须存储在任何必须的参数。注意，有些块不需要任何参数。最后，为了计算梯度，块必须具有反向传播函数。在定义我们自己的块时，由于自动微分提供了一些后端实现，我们只需要考虑前向传播函数和必须的参数。

首先，我们回顾一下多层感知机。         
我们通过`nn.Sequential`来实现我们的模型，层的执行顺序是作为参数传递的。简而言之，`nn.Sequential`定义了一种特殊的`Module`，即在PyTorch中表示一个块的类，它维护了一个由`Module`组成的有序列表。注意，两个全连接层都是`Linear`类的实例，`Linear`类本身就是`Module`的子类。        
到目前为止，我们一直在用`net(X)`来获取模型的输出，这实际上是`net.__call__(X)`的简写。这个前向传播函数非常简单，它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
X, net(X)

(tensor([[0.7072, 0.2686, 0.6287, 0.4289, 0.5911, 0.8047, 0.3181, 0.8521, 0.4765,
          0.4142, 0.9483, 0.7728, 0.4051, 0.5280, 0.1123, 0.6019, 0.7756, 0.2588,
          0.3354, 0.3994],
         [0.2577, 0.8380, 0.2186, 0.9706, 0.2038, 0.2379, 0.0894, 0.9215, 0.6541,
          0.2501, 0.9599, 0.1179, 0.0046, 0.1727, 0.2171, 0.8860, 0.8009, 0.6262,
          0.6129, 0.8566]]),
 tensor([[ 0.2694, -0.2003, -0.3185,  0.0256,  0.1815, -0.2071,  0.0973, -0.0400,
          -0.1360,  0.3118],
         [ 0.2541, -0.2231, -0.3391,  0.0699,  0.0134, -0.1328,  0.1097, -0.1651,
          -0.1218,  0.2265]], grad_fn=<AddmmBackward0>))

### 自定义块   
注意一些关键细节： 首先，我们定制的`__init__`函数通过`super().__init__()` 调用父类的`__init__`函数,省去了重复编写模版代码的痛苦。然后，我们实例化两个全连接层，分别为`self.hidden`和`self.out`。注意，除非我们实现一个新的运算符，否则我们不必担心反向传播函数或参数初始化，系统将自动生成这些。

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的类，然后在每次调用正向传播函数时调用这些层

In [4]:
net = MLP()
net(X)

tensor([[-0.0943, -0.1981,  0.0222, -0.1430,  0.1750, -0.0149, -0.2107, -0.1140,
          0.1048,  0.1997],
        [-0.2130, -0.2295, -0.0299, -0.1918,  0.1972,  0.0358, -0.2875, -0.1658,
         -0.0042,  0.1552]], grad_fn=<AddmmBackward0>)

### 顺序块
`MySequential`类提供了与默认`Sequential`类相同的功能。我们只需要定义两个关键函数。
1. 一种将块逐个追加到列表中的函数
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
        
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1583,  0.0567, -0.1712,  0.0428, -0.0491,  0.2620,  0.0086, -0.1324,
         -0.0364,  0.0496],
        [-0.0985,  0.0332, -0.0290,  0.0436, -0.0187,  0.1100,  0.0531, -0.1290,
         -0.0374,  0.1027]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
X, net(X)

(tensor([[0.7072, 0.2686, 0.6287, 0.4289, 0.5911, 0.8047, 0.3181, 0.8521, 0.4765,
          0.4142, 0.9483, 0.7728, 0.4051, 0.5280, 0.1123, 0.6019, 0.7756, 0.2588,
          0.3354, 0.3994],
         [0.2577, 0.8380, 0.2186, 0.9706, 0.2038, 0.2379, 0.0894, 0.9215, 0.6541,
          0.2501, 0.9599, 0.1179, 0.0046, 0.1727, 0.2171, 0.8860, 0.8009, 0.6262,
          0.6129, 0.8566]]),
 tensor(0.0292, grad_fn=<SumBackward0>))

混合搭配各种组合块的方法:

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1304, grad_fn=<SumBackward0>)